#Installing required dependencies

In [1]:
!pip install transformers sentence-transformers chromadb datasets accelerate faiss-cpu torch huggingface_hub python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s

In [ ]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv(".env")

# Loading Data

In [2]:
from datasets import load_dataset
from huggingface_hub import login

# Access the environment variables
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

# Load dataset
ds = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")["train"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.40k [00:00<?, ?B/s]

preprocessed_data.json:   0%|          | 0.00/222M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/945278 [00:00<?, ? examples/s]

## Initialize ChromaDB storage

In [3]:
import chromadb


# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Persistent storage

# Create a collection
collection = chroma_client.get_or_create_collection(name="customer_support")

## Initializing Embedding Model

In [4]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(embedding_model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Adding Data to the ChromaDB Collection

In [6]:
for i, data in enumerate(ds):
    query_text = data["input"]
    response_text = data["output"]
    embedding = embedding_model.encode(query_text).tolist()  # Convert to list for ChromaDB

    collection.add(
        ids=[str(i)],  # Unique ID for each entry
        embeddings=[embedding],  # Vector representation
        metadatas=[{"input": query_text, "output": response_text}]
    )
    
print("ChromaDB populated with customer support queries!")

ChromaDB populated with customer support queries!


## Load LLAMA2 Model

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load LLaMA 2 model & tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Change this based on your resources
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

print("LLaMA 2 model loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LLaMA 2 model loaded successfully!


In [17]:
# Cache to store the last 5 queries and responses
cache = []

def create_response(user_query):

    # Step 1: Encode the user query
    query_embedding = embedding_model.encode(user_query).tolist()

    # Step 2: Retrieve the most relevant query-response pairs
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5  # Retrieve top 5 similar queries
    )

    # Step 3: Format retrieved context
    retrieved_context = "\n".join(
        [f"User: {res['input']}\nSupport: {res['output']}" for res in results["metadatas"][0]]
    )

    global cache

    # Construct the cache context from the most recent to the oldest
    cache_context = "\n".join([f"User: {item['query']}\nSupport: {item['response']}" for item in reversed(cache)])

    # Step 4: Construct prompt for LLaMA 2
    prompt = f"""
    You are a helpful and accurate customer support AI.

    Your primary goal is to assist users by providing clear, concise, and correct answers based on the information available.
    Leverage the following sources to generate your response:
    1. Retrieved past responses from similar queries.
    2. Recent user interactions and responses.
    3. Context from the previous conversation, if relevant.

    Guidelines:
    - Only provide a direct answer to the new query. Avoid repeating the prompt or giving unnecessary explanations.
    - Base your response solely on the information available. Do not use internal knowledge or make assumptions.
    - If the answer is unknown, clearly state that you do not know and suggest relevant questions that the user can ask.
    - If the user asks for clarification or a follow-up on a previous response, make use of the last message to maintain continuity and context.


    Recent Interactions:
    {cache_context}

    Retrieved Context:
    {retrieved_context}

    New User Query: {user_query}
    Support AI Response: """

    # Step 5: Generate response with LLaMA 2
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output_tokens = model.generate(**inputs, max_length=1000)
    response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # To avoid if repetitoin of prompt in response.
    response = response.replace(prompt, "").strip()

    # Add the current query and response to the cache
    cache.append({"query": user_query, "response": response})

    # Ensure the cache does not exceed 5 items
    if len(cache) > 5:
        cache.pop(0)  # Remove the oldest entry

    return response

In [18]:
# user_query = "I ordered a laptop, but it arrived with a broken screen. What should I do?"
# response = create_response(user_query)
# print(response)

Sorry to hear that your laptop arrived with a broken screen. Can you please provide the order number and confirm the shipping details? We will assist you with the replacement or repair of the screen.


In [21]:
# user_query = "I don't understand why you need my order details. what you willdo after that with it?"
# response = create_response(user_query)
# print(response)

Hi! Thank you for reaching out to us. We need your order details to assist you with your query. We will use this information to verify your order and provide you with the appropriate next steps. If you have any questions or concerns, please feel free to ask.


In [22]:
!pip install flask flask-ngrok

In [23]:
pip install pyngrok

In [25]:
from pyngrok import ngrok
port_no = 5000
ngrok_token = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(ngrok_token)
public_url =  ngrok.connect(port_no).public_url

In [26]:
public_url

'https://5128-34-19-2-115.ngrok-free.app'

In [27]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok  # Allows Flask to work on Colab

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok to expose the server

@app.route('/generate_response', methods=['POST'])
def generate_response():
    data = request.json
    if not data or "query" not in data:
        return jsonify({"error": "Missing 'query' parameter"}), 400  # Return error if 'query' is missing
    else:
      print(data)
    user_query = data["query"]
    response = create_response(user_query)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5128-34-19-2-115.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
{'query': 'I need help resetting my password.'}


INFO:werkzeug:127.0.0.1 - - [29/Mar/2025 20:32:08] "POST /generate_response HTTP/1.1" 200 -


{'query': 'I didn’t receive the reset link.'}


INFO:werkzeug:127.0.0.1 - - [29/Mar/2025 20:33:00] "POST /generate_response HTTP/1.1" 200 -


{'query': 'My cat chewed my phone charger. Is this covered under warranty?'}


INFO:werkzeug:127.0.0.1 - - [29/Mar/2025 20:33:58] "POST /generate_response HTTP/1.1" 200 -


{'query': 'Why did you suggest contacting support?'}


INFO:werkzeug:127.0.0.1 - - [29/Mar/2025 20:34:33] "POST /generate_response HTTP/1.1" 200 -
